In [ ]:
import base64
import hmac
import time
import requests
import urllib.request
from dotenv import load_dotenv
import os
import json
from hashlib import sha1 as sha

In [ ]:
# load environment variables from .env file
load_dotenv("local.env")

In [ ]:
host = "http://api.velocityweather.com/v1"
access_key = os.getenv("BARON_KEY")
if access_key is None:
    raise ValueError("Missing BARON_KEY in local.env file")
access_key_secret = os.getenv("BARON_SECRET")
if access_key_secret is None:
    raise ValueError("Missing BARON_SECRET in local.env file")

In [ ]:
def sign(string_to_sign, secret):
    hmac_sha1 = hmac.new(
        # Convert secret to bytes using UTF-8
        secret.encode('utf-8'),
        # Convert input string to bytes using UTF-8
        string_to_sign.encode('utf-8'),
        sha                        # Use SHA1 hash algorithm
    )

    # Get the binary digest
    hmac_digest = hmac_sha1.digest()

    # Encode to base64
    base64_encoded = base64.b64encode(hmac_digest).decode('utf-8')

    # Replace characters to make URL safe
    signature = base64_encoded.replace('/', '_').replace('+', '-')

    return signature


def sign_request(url, key, secret):
    """ Returns signed url
    """

    ts = str(int(time.time()))
    sig = sign(key + ":" + ts, secret)
    q = '?' if url.find("?") == -1 else '&'
    url += "%ssig=%s&ts=%s" % (q, sig, ts)
    return url

In [ ]:
def fetch_url(url):
    try:
        # Make the request with gzip compression support
        response = requests.get(url, headers={"Accept-Encoding": "gzip"})
        response.raise_for_status()
        return response.json()
    except requests.RequestException as e:
        print(f"[VelocityWeather] Request failed: {e}")
        return None

In [ ]:
def get_product_instance(product, product_config):
    """
    Retrieve product instance metadata from the Velocity Weather API.
    
    This function fetches the latest product instance information for a given
    product and configuration. It uses request signing for authentication and
    handles error cases appropriately.
    
    Args:
        product (str): The product identifier (e.g., "fire-tracker-us")
        product_config (str): The product configuration (e.g., "Standard-Mercator")
        
    Returns:
        dict or None: The product instance metadata if successful, None if the request fails
        
    Example:
        >>> instance_data = get_product_instance("fire-tracker-us", "Standard-Mercator")
        >>> if instance_data:
        ...     print(instance_data[0]['time'])
    """
    # Construct the metadata URL
    meta_url = f"{host}/{access_key}/meta/tiles/product-instances/{product}/{product_config}"
    
    # Sign the request for authentication
    signed_url = sign_request(meta_url, access_key, access_key_secret)
    
    # Fetch and validate the response
    data = fetch_url(signed_url)
    if not data:
        print("Error: Product meta info not found.")
    return data

In [ ]:
def get_instance_time(product, config):
    data = get_product_instance(product, config)
    if not data:
        return None

    instance_time = data[0]['time'] 
    return instance_time

In [ ]:
def request_geotiff_url(product, product_config, image_bounds):
    """
    Build the Baron API URL for a GeoTIFF request.
    """
    product_instance_time = get_instance_time(product, product_config)
    if not product_instance_time:
        print("Error: Product instance time not found.")
        return None

    geotiff_url = f"{host}/{access_key}/geotiff/{product}/{product_config}/{product_instance_time}?&BBOX={image_bounds}"
    
    geotiff_url = sign_request(geotiff_url, access_key, access_key_secret)
    print(geotiff_url)

    return(geotiff_url)

In [ ]:
def request_geotiff_image(product, product_config, image_bounds):
    """ 
    Requests a GeoTIFF image and saves it to disk in the current directory.
    """
    geotiff_url = request_geotiff_url(product, product_config, image_bounds)
    if geotiff_url is None:
        print('Failed to get GeoTIFF URL')
        return
    try:
        response = urllib.request.urlopen(geotiff_url)
    except urllib.error.HTTPError as e:
        print('HTTP status code:', e.code)
        print('content:')
        print(e.read())
        return
    assert response.code == 200

    content = response.read()
    print(f"GeoTiff content: {content}")
    geotiff_json = json.loads(content)

    geotiff_image_url = geotiff_json['source']

    return geotiff_image_url

In [ ]:
texas_bound_box = [-106.645646, 36.500704, -93.508292, 25.837164]
texas_bbox_str = "-106.645646,36.500704,-93.508292,25.837164"
usa_bound_box = [-125.859375, 25.618963, -63.193359, 49.378416]
whole_world_bound_box = [-90, -180, 90, 180]

In [ ]:
# product =  'north-american-radar'
# product = 'baron-hires-haildiameter' # 'flash-flood-risk' 
product = 'C39-0x03A1-0' # Damaging Wind Path
# 'baron-hires-haildiameter' #'fire-tracker-us' #'north-american-radar' #'fspc-day2-outlook' #  'C39-0x0302-0' #  #'C39-0x0355-0' # 
product_config = 'Standard-Geodetic' # 
image_size_in_pixels = [2048, 2048]
image_bounds = texas_bound_box


In [ ]:
geotiff_image_url = request_geotiff_image(product, product_config, texas_bbox_str)

In [ ]:
geotiff_image_url

In [ ]:
from datetime import datetime

current_datetime = datetime.now().strftime("%Y%m%d_%H%M%S")
current_datetime

In [ ]:
geotiff_image_filename = f'./{product}_{product_config}_{current_datetime}.tiff'
if geotiff_image_url:
    response = requests.get(geotiff_image_url)
    if response.status_code == 200:
        filename = f'./wms_img_{product}_{product_config}.tiff'
        with open(geotiff_image_filename, 'wb') as f:
            f.write(response.content)
        print(f"GeoTIFF image downloaded and saved as {geotiff_image_filename}")
    else:
        print(f"Failed to download GeoTIFF image. Status code: {response.status_code}")
else:
    print("geotiff_image_url is not set.")

In [ ]:
import rasterio
import geopandas as gpd
from rasterio.features import shapes

with rasterio.open(geotiff_image_filename) as src:
    image = src.read(1)
    mask = image != src.nodata
    results = (
        {"properties": {"value": v}, "geometry": s}
        for s, v in shapes(image, mask=mask, transform=src.transform)
    )
    geoms = list(results)

In [ ]:
gdf = gpd.GeoDataFrame.from_features(geoms, crs=src.crs)

In [ ]:
print(f"Head= {gdf.head()}")
print(f"CRS= {gdf.crs}")
print(f"Shape= {gdf.shape}")
gdf.head()

In [ ]:
gdf.to_file(f"{product}_{product_config}_{current_datetime}.geojson", driver="GeoJSON")

### Get weather geometries that are inside Texas

In [ ]:
texas_gdf = gpd.read_file("texas.geojson")

#### Get geometry with 50 mile buffer around Texas

In [ ]:
# Convert to a projected CRS, e.g., EPSG:3083 (Texas Centric Albers Equal Area)
texas_proj = texas_gdf.to_crs(epsg=3083)

# 50 miles in meters (1 mile ≈ 1609.34 meters)
buffered_texas_proj = texas_proj.buffer(50 * 1609.34)

# Create a new GeoDataFrame for the buffered geometry
buffered_texas_gdf = gpd.GeoDataFrame(geometry=buffered_texas_proj, crs=texas_proj.crs)

# Optionally convert back to EPSG:4326
buffered_texas_gdf = buffered_texas_gdf.to_crs(epsg=4326)

In [ ]:
buffered_texas_gdf.to_file(f"texas_buffered.geojson", driver="GeoJSON")

#### Get weather geometries that are inside Texas 50 mile buffer

In [ ]:
from shapely.geometry import Polygon, MultiPolygon

# Ensure both GeoDataFrames use the same CRS
if gdf.crs != buffered_texas_gdf.crs:
    gdf = gdf.to_crs(buffered_texas_gdf.crs)

# Get the union of all Texas geometries
texas_union = texas_gdf.geometry.union_all()

# Select geometries in gdf that are within Texas
gdf_within_texas = gdf[gdf.geometry.within(texas_union)]

In [ ]:
print(f"Head=\n{gdf_within_texas.head()}")
print(f"CRS={gdf_within_texas.crs}")
print(f"Shape= {gdf_within_texas.shape}")

In [ ]:
gdf_within_texas.to_file(f"tx_{product}_{product_config}_{current_datetime}_within.geojson", driver="GeoJSON")

## Visualize on map

In [ ]:
import leafmap
hc_lat = 29.85722627925764
hc_long = -95.39202050686889
m = leafmap.Map(center=[hc_lat, hc_long], zoom=7,
                epsg="4326")

In [ ]:
m.add_gdf(gdf_within_texas, layer_name=product, style={"color": "blue", "weight": 2})

In [ ]:
m.add_gdf(texas_gdf, layer_name="Texas", style={"color": "red", "weight": 2})

In [ ]:
m